In [1]:
import pandas as pd
from haversine import haversine

In [37]:
city_list=["광주", "울산","전주","창원"]
i=0
fire_incident_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+" 화재 정보.csv")
accident_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+"_교통사고_통계.csv")
school_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+" 초중고.csv")
bus_station_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+"시_버스 정류장.csv")
gas_station_df=pd.read_csv(city_list[i]+" data 경도 위도 매핑/"+city_list[i]+"_충전소_주유소.csv")
weight_df=pd.DataFrame([[0]*7]*len(gas_station_df), columns=['주유소명', '버정거리총합', '상호개수', '교통사고건수','보호구역개수','화재발생건수','총 가중치'])
weight_df['주유소명']=gas_station_df['상호'][:]
## 추가추가

In [38]:
fire_incident_df.head(3)

,조사서번호,시도,시군구,읍면동,주소,위도,경도
0,'170101192708682,광주광역시,북구,신용동,광주광역시 북구 신용동,35.213635,126.863054
1,'170101194942936,광주광역시,북구,용봉동,광주광역시 북구 용봉동,35.213635,126.863054
2,'170101214940902,광주광역시,광산구,장덕동,광주광역시 광산구 장덕동,35.213635,126.863054


In [39]:
accident_df.head(3)

,시군구,도로명,주소,사고건수,사망자수,중상자수,경상자수,부상신고자수,위도,경도
0,광주광역시 동구,필문대로,광주광역시 동구 필문대로,4,1,1,3,0,35.158095,126.928527
1,광주광역시 동구,참판로,광주광역시 동구 참판로,3,0,0,3,0,35.160837,126.927285
2,광주광역시 동구,화산로350번길,광주광역시 동구 화산로350번길,1,0,0,1,0,35.115439,126.932745


In [40]:
bus_station_df.head(3)

,NODE_NM,위도,경도
0,방림터널사거리,35.130698,126.918418
1,방림터널사거리,35.130785,126.918429
2,송림아파트,35.130956,126.923305


In [41]:
gas_station_df.head(3)

,지역,상호,타입,주소,위도,경도
0,광주광역시,(주)대원 강변주유소,주유소,광주 서구 천변좌하로 142 (치평동),35.152433,126.856406
1,광주광역시,반디석유시청점,주유소,광주 서구 천변좌하로 160 (치평동),35.153657,126.836214
2,광주광역시,(주)아프리카주유소,주유소,광주광역시 서구 천변좌하로 168 (치평동),35.155775,126.837847


## 가산

가능 : 타 주유소, 교통량(교통 사고), 버스 정류장

working on : 차고지 위치

In [42]:
# 거리 총합 리스트
def 거리총합리스트(data1, data2, name_col):
    # distance_list=[]

    for i in range(len(data1)):
        total=0
        for j in range(len(data2)):
            total+=haversine((data1.loc[i]['위도'],data1.loc[i]['경도']),(data2.loc[j]['위도'],data2.loc[j]['경도']), unit = 'km')
        # distance_list.append((data1.loc[i][name_col],total))
        weight_df['버정거리총합'][i]=total
    

In [43]:
# p 반경 이내 개수 리스트(타 주유소)
def 반경이내_개수_리스트(p, data1, data2, name_col):
    count_list=[]
    for i in range(len(data1)):
        count=0
        for j in range(len(data2)):
            dist=haversine((data1.loc[i]['위도'],data1.loc[i]['경도']),(data2.loc[j]['위도'],data2.loc[j]['경도']), unit = 'km')
            if dist<=p:
                # 교통사고 데이터 사고 건수로 count!
                if ('사고건수' in data2.columns):
                    count+=data2['사고건수']
                else:
                    count+=1
        #count_list.append((data1.loc[i][name_col],count-1)) 상호명 매핑 화긴~
  
        if count==0:
            count_list.append(count)
        else:
            count_list.append(count-1)# 자기 자신 제외

    return count_list
   

In [ ]:
""" 이건 km
거리총합리스트(gas_station_df, bus_station_df, '상호')
weight_df['버정거리총합']=max(weight_df['버정거리총합'])-weight_df['버정거리총합']
weight_df
""" # 이건 개수
버정거리총합 = 반경이내_개수_리스트(2, gas_station_df, bus_station_df,'상호')
weight_df['버정거리총합']=버정거리총합
weight_df

In [ ]:
상호개수 = 반경이내_개수_리스트(2, gas_station_df, gas_station_df,'상호')
weight_df['상호개수']=상호개수
weight_df

In [ ]:
######이거 수정 : 도로당 회수 있음, 거리 기준으로 함수 변경-> 거리가 작을수록, 회수가 클 수록 좋은,,데,,,
교통사고건수=반경이내_개수_리스트(2, gas_station_df, accident_df,'상호')
weight_df['교통사고건수']=교통사고건수
weight_df

## 감점

가능 : 어린이집, 유초중고(2km)

working on : 화재

In [ ]:
fire_incident_df.head(3)

In [ ]:
gas_station_df.head(3)

In [ ]:
school_df.head(3)

In [ ]:
kinder_df.head(3)

In [ ]:
#유치원,,
#보호구역개수=반경이내_개수_리스트(2, gas_station_df, kinder_df, '상호')
#weight_df['보호구역개수']=보호구역개수

In [ ]:
보호구역개수=반경이내_개수_리스트(2, gas_station_df, school_df, '상호')
weight_df['보호구역개수']=보호구역개수
weight_df

In [ ]:
화재발생건수=반경이내_개수_리스트(2, gas_station_df, fire_incident_df, '상호')
weight_df['화재발생건수']=화재발생건수
weight_df

## 스케일링

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
print(minMaxScaler.fit(weight_df.iloc[:,[1,2,3,4,5]]))
weight_df_stdScaled = minMaxScaler.transform(weight_df.iloc[:,[1,2,3,4,5]])
weight_df_stdScaled=pd.DataFrame(weight_df_stdScaled, columns=["버정거리총합","상호개수","교통사고건수","보호구역개수","화재발생건수"])
weight_df_stdScaled["주유소명"]=weight_df["주유소명"][:]
weight_df_stdScaled

In [ ]:
weight_df_stdScaled.to_csv(city_list[i]+" 스케일링완료.csv")

# ########## 여기까지1 - 스케일링 완, 가중치 미완(lr용) ############

## 가중치 매핑 함수
새로운 데이터 프레임 만들어서 '상호'와 가중치1,2,3,4,5 넣고 총합 마지막에 하고 sort해서 [:20] 출력

### 가산
주유소_count_list=반경이내_개수_리스트(2, gas_station_df, gas_station_df,'상호')

distance_list=거리총합리스트(gas_station_df, bus_station_df, '상호')

교통사고_count_list=반경이내_개수_리스트(2, gas_station_df, accident_df,'상호')
### 감점
유치원_count_list=반경이내_개수_리스트(2, gas_station_df, kinder_df, '상호')

유초중고_count_list=반경이내_개수_리스트(2, gas_station_df, school_df, '상호')

화재_count_list=반경이내_개수_리스트(2, gas_station_df, fire_incident_df, '상호')

In [ ]:
# **주유소 순서 헤치지 말 것**
#columns=['버정거리총합', '상호개수', '교통사고건수','보호구역개수','화재발생건수',주유소명'])
def 가중치추가(df, ratio_list):
    for i in range(0,len(df.loc[0])-2):
        df['총 가중치']+=df.iloc[:,i]*ratio_list[i]# weight_df는 주유소명부터, ratio_list는 버정거리총합부터 시작하므로
    


In [ ]:
weight_df_stdScaled["총 가중치"]=0 # 혹시 모를 사고에 대비
ratio_list=[0.001,0.1,0.1,-0.1,-0.1] # 뒤에 두개 음수 필수
가중치추가(weight_df_stdScaled, ratio_list)
weight_df_stdScaled.head(5)

In [ ]:
weight_df_stdScaled_sorted=weight_df_stdScaled.sort_values(by=["총 가중치", "주유소명"], ascending=[False, False]) 
weight_df_stdScaled_sorted

In [ ]:
weight_df_sorted.to_csv(city_list[2]+" 가중치.csv")

# ########## 여기까지2 - 스케일링, 가중치 모두 완 (f1용)

## 정확도 나타내기
가중치가 높은 상위 20개 중 수소 충전소가 있는 개수
- 전체 충전소 개수 중 맞춘 개수?

In [ ]:
# 예시예시 10개!! 실제로는 불러와야겠죠,,,
guess_df=pd.DataFrame([[0]*2]*10, columns=['주유소명','어쩌구'])
# 찐 걍 예시

guess_df['주유소명']=gas_station_df['상호'][:10]
guess_df

In [ ]:
# 예시 친구들을 전체 주유소에 대입
labels_df=pd.DataFrame([[0]*2]*len(weight_df_sorted), columns=['주유소명','label'])
labels_df['주유소명']=gas_station_df['상호']
for i in range(len(guess_df)):
    if labels_df['주유소명'][i]==guess_df['주유소명'][i]:
        labels_df['label'][i]+=1

In [ ]:
# weight_df_sorted 상위 10개 label에 1 넣음
eval_df=pd.DataFrame([[0]*2]*len(weight_df_sorted), columns=['주유소명','label'])
eval_df['주유소명']=weight_df_sorted['주유소명'][:]
eval_df['label'][:len(guess_df)]=1#일단 그냥 추측 개수 같게 했다,,

In [ ]:
# 전체 충전소 개수 중 guess의 개수 안에 있는 충전소 개수
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
eval_df['label']=0
# eval_df.columns=['주유소명','label']
for i in range(len(guess_df)):
    if guess_df['주유소명'][i]==eval_df['주유소명'][i]:
        eval_df['label'][i]+=1

eval_df.sort_values(by=["주유소명"], ascending=[False]) 
guess_df.sort_values(by=["주유소명"], ascending=[False]) 
# 원래 친구
labels=labels_df['label']
# 추정한 친구
guesses=eval_df['label']
guesses

In [ ]:
print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses))
print(precision_score(labels, guesses))
print(f1_score(labels, guesses))

## 동 매핑

동이랑 통행속도 매핑

gas_station_df['주소']-도로 이름 나오니까 그냥 사용

travel_speed['도로명'], ['속도']

In [ ]:
# 그냥 대충 만든 거
travel_speed=pd.DataFrame([['혁신로', 20],['기지로', 30],['쑥고개로', 10],['권삼득로', 33]], columns=['도로명','속도'])
travel_speed

In [ ]:
# 가중치 데이터 프레임
weight_df=pd.DataFrame([[0]*5]*len(gas_station_df), columns=['주유소_이름', '주소','통행속도_w', 'a', 'b'])
weight_df['주유소_이름']=gas_station_df['상호'][:]
weight_df['주소']=gas_station_df['주소'][:]
weight_df

In [ ]:
for i in range(len(travel_speed)):
    for j in range(len(gas_station_df)):
        if travel_speed.loc[i]['도로명'] in gas_station_df.loc[j]['주소']:
            # gas_station_df.loc[j]['상호']
            # 하는 중
            # weight_df.iloc['주유소_이름']['통행속도_w']+=1
weight_df